In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
import pickle

# Preprocessing
- Random sampling
- Categorical values
- Missing values
- Feature extraction
- Scale variables

In [2]:
train_data = pd.read_parquet('train_data.parquet')

In [3]:
train_data.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
count,5.485466e+06,5.531451e+06,5.531451e+06,5.529435e+06,5.531451e+06,4.510907e+06,5.529435e+06,5.529435e+06,791314.000000,3.873055e+06,...,1.946990e+05,1.946990e+05,5.429903e+06,5.490819e+06,5.429903e+06,944408.000000,5.429903e+06,5.490724e+06,5.429903e+06,5.531451e+06
mean,6.581885e-01,1.514543e-01,1.225995e-01,6.105151e-01,7.632776e-02,2.264676e-01,5.831698e-02,1.302262e-01,0.184966,1.544140e-01,...,1.424414e-02,1.646144e-01,1.750432e-01,2.634348e-02,1.615358e-01,0.390781,1.749181e-01,5.131478e-02,6.084983e-02,2.490972e-01
std,2.441398e-01,2.705955e-01,2.131456e-01,4.029192e-01,2.221323e-01,1.925726e-01,1.984337e-01,2.343981e-01,0.228140,2.129113e-01,...,9.569677e-02,2.670328e-01,3.696215e-01,1.442015e-01,3.456256e-01,0.236142,3.695211e-01,1.791743e-01,1.889250e-01,4.324903e-01
min,-4.589548e-01,5.026190e-09,-7.588799e+00,9.192280e-09,1.534223e-09,-6.271321e-01,5.566545e-10,6.285293e-09,-0.000454,1.154550e-07,...,1.078787e-08,3.307923e-08,3.767347e-10,3.725073e-09,1.650100e-10,-0.014539,5.549692e-09,2.500991e-09,1.226024e-09,0.000000e+00
25%,4.803307e-01,4.528464e-03,8.863645e-03,1.053313e-01,2.895934e-03,1.272588e-01,2.873244e-03,5.227570e-03,0.037516,4.227546e-02,...,2.532470e-03,3.517452e-03,3.027212e-03,2.555848e-03,3.026087e-03,0.199399,3.028116e-03,2.752895e-03,3.028347e-03,0.000000e+00
50%,6.942950e-01,9.056902e-03,3.132968e-02,8.143328e-01,5.782230e-03,1.639082e-01,5.746725e-03,9.777229e-03,0.120519,8.851244e-02,...,5.069830e-03,7.037814e-03,6.053010e-03,5.110523e-03,6.051590e-03,0.382136,6.053151e-03,5.508129e-03,6.053247e-03,0.000000e+00
75%,8.648159e-01,2.366407e-01,1.259019e-01,1.002403e+00,8.660590e-03,2.581017e-01,8.615665e-03,1.550507e-01,0.250869,1.843206e-01,...,7.573434e-03,5.015469e-01,9.080455e-03,7.663697e-03,9.078914e-03,0.559307,9.076287e-03,8.260448e-03,9.080930e-03,0.000000e+00
max,1.010000e+00,5.389619e+00,1.324060e+00,1.010000e+00,3.256284e+00,5.482888e+00,8.988807e+00,1.625262e+00,4.191119,1.011162e+01,...,1.009998e+00,3.005383e+00,1.010000e+00,1.010000e+00,1.339910e+00,2.229368,1.010000e+00,1.343331e+00,4.827630e+00,1.000000e+00


## Random Sampling
We select 10% of the clients

In [4]:
clients = train_data['customer_ID'].unique()
n = int(0.1*len(clients))
selected_clients = np.random.choice(clients, n, replace=False)

In [5]:
train = train_data[train_data.customer_ID.isin(selected_clients)]
print("Original train dataset shape was:", train_data.shape)
print("Random sample dataset now is: ", train.shape)

Original train dataset shape was: (5531451, 191)
Random sample dataset now is:  (553697, 191)


We verify our sample represents correctly the original train set

In [6]:
train.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
count,549074.000000,5.536970e+05,553697.000000,5.535090e+05,5.536970e+05,451255.000000,5.535090e+05,5.535090e+05,79158.000000,3.890390e+05,...,1.939100e+04,1.939100e+04,5.434620e+05,5.495350e+05,5.434620e+05,94521.000000,5.434620e+05,5.495190e+05,5.434620e+05,553697.000000
mean,0.656129,1.535190e-01,0.123318,6.202384e-01,7.954329e-02,0.226380,6.033329e-02,1.324444e-01,0.185790,1.547614e-01,...,1.435675e-02,1.649387e-01,1.787974e-01,2.579877e-02,1.645867e-01,0.395964,1.786730e-01,5.444669e-02,6.204691e-02,0.250289
std,0.244189,2.702014e-01,0.210495,4.017786e-01,2.279593e-01,0.194216,2.050364e-01,2.342118e-01,0.233211,2.084194e-01,...,9.647680e-02,2.656294e-01,3.793010e-01,1.427060e-01,3.486367e-01,0.236682,3.791972e-01,1.862663e-01,1.917907e-01,0.433180
min,-0.375840,3.785832e-08,-0.799453,2.482187e-07,2.066058e-08,-0.491539,3.516009e-08,6.285293e-09,-0.000251,6.194853e-07,...,2.534351e-08,9.859704e-08,6.598718e-08,1.807036e-08,1.650100e-10,-0.014228,3.909845e-08,1.325409e-08,2.246565e-08,0.000000
25%,0.479678,4.538830e-03,0.008864,1.048189e-01,2.898846e-03,0.126996,2.878733e-03,5.214345e-03,0.036873,4.235244e-02,...,2.516954e-03,3.516602e-03,3.020772e-03,2.559071e-03,3.025374e-03,0.205362,3.031009e-03,2.759574e-03,3.028541e-03,0.000000
50%,0.694792,9.080932e-03,0.031372,8.143299e-01,5.783508e-03,0.163844,5.751648e-03,9.780595e-03,0.118108,8.934197e-02,...,4.989270e-03,7.062660e-03,6.050527e-03,5.116322e-03,6.050216e-03,0.388025,6.060432e-03,5.526314e-03,6.043850e-03,0.000000
75%,0.863288,2.368908e-01,0.125867,1.002400e+00,8.668484e-03,0.258857,8.627107e-03,1.561719e-01,0.249029,1.863929e-01,...,7.507271e-03,5.016676e-01,9.083773e-03,7.663543e-03,9.074696e-03,0.566239,9.081175e-03,8.294351e-03,9.083920e-03,1.000000
max,1.009999,5.357074e+00,1.324059,1.010000e+00,2.756794e+00,3.502517,7.510052e+00,1.625262e+00,4.186889,9.321696e+00,...,1.009998e+00,2.008491e+00,1.010000e+00,1.009999e+00,1.252589e+00,1.837051,1.010000e+00,1.343293e+00,3.279571e+00,1.000000


## Categorical Values

In [7]:
data_types = { col: train[col].dtypes for col in train.columns}
non_numeric_cols = [d for d in data_types.keys() if data_types[d] == 'O' and d not in ['customer_ID', 'S_2']]
non_numeric_cols

['D_63', 'D_64']

In [8]:
# categorical columns D_63 and D_64
for col in non_numeric_cols:
    le = preprocessing.LabelEncoder()
    train[col] = le.fit_transform(train[col])

## Missing values

In [9]:
# imputation of missing values
X = train.set_index(['customer_ID', 'S_2'])
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
train_new = pd.DataFrame(imp.transform(X), index = X.index, columns= X.columns)
train_new = train_new.reset_index()
train_new

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2017-03-11,0.876615,0.001469,0.001472,0.810796,0.005955,0.226382,0.008401,0.004826,...,0.014357,0.164939,0.003652,0.004395,0.002179,0.395966,0.001196,0.002738,0.000443,0.0
1,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2017-04-11,0.887385,0.004229,0.002125,0.819078,0.004521,0.226382,0.008008,0.007282,...,0.014357,0.164939,0.002876,0.000840,0.009347,0.395966,0.003332,0.009278,0.004498,0.0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2017-05-12,0.884522,0.003399,0.003495,0.818569,0.007541,0.226382,0.007537,0.000859,...,0.014357,0.164939,0.003084,0.000304,0.000891,0.395966,0.002980,0.009026,0.002959,0.0
3,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2017-06-10,0.852514,0.006877,0.007627,0.819987,0.009290,0.226382,0.003959,0.007532,...,0.014357,0.164939,0.007998,0.003724,0.005225,0.395966,0.005043,0.001033,0.005859,0.0
4,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2017-07-12,0.891656,0.000802,0.009997,0.811041,0.003540,0.226382,0.009034,0.000626,...,0.014357,0.164939,0.009134,0.005561,0.006398,0.395966,0.002139,0.003595,0.005110,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553692,fffd765f1f5fa550f044acb297092807227f4742fa11bb...,2017-11-28,0.686476,0.211116,0.055055,1.004938,0.005941,0.141466,0.008663,0.002887,...,0.000364,0.009860,0.008247,0.000969,0.001880,0.395966,0.009615,0.001447,0.005410,0.0
553693,fffd765f1f5fa550f044acb297092807227f4742fa11bb...,2017-12-03,0.684926,0.361464,0.060740,1.000378,0.002943,0.145406,0.008054,0.006243,...,0.003319,0.003286,0.003205,0.000724,0.002089,0.395966,0.004947,0.005034,0.007218,0.0
553694,fffd765f1f5fa550f044acb297092807227f4742fa11bb...,2018-01-02,0.709665,0.330995,0.081282,1.003688,0.007902,0.131936,0.008661,0.010533,...,0.002360,0.009094,0.002246,0.005936,0.006764,0.395966,0.000953,0.003639,0.002407,0.0
553695,fffd765f1f5fa550f044acb297092807227f4742fa11bb...,2018-02-24,0.733438,0.149128,0.034771,1.006038,0.001128,0.125726,0.002577,0.010326,...,0.001384,0.002081,0.001148,0.004779,0.005688,0.395966,0.001870,0.009649,0.004466,0.0


## Feature extraction

In [10]:
#mean_df = train_new.drop(columns='S_2').groupby('customer_ID').diff(axis = 0).drop(columns = ['target']).fillna(0)
#mean_df['customer_ID'] = train_new['customer_ID']
#mean_df = mean_df.groupby('customer_ID').mean()
#max_df = train_new.drop(columns=['S_2', 'target']).groupby('customer_ID').max()
mean_df = train_new.drop(columns=['S_2', 'target']).groupby('customer_ID').mean()
last_df = train_new.groupby('customer_ID').last().drop(columns = ['S_2'])

In [11]:
df = pd.concat([mean_df, last_df], axis=1)
df

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.878454,0.004181,0.004386,0.815677,0.006621,0.226382,0.006842,0.005493,0.18579,0.154763,...,0.014357,0.164939,0.007383,0.006623,0.000964,0.395966,0.002202,0.000834,0.003444,0.0
000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370150fc2f8bdca6202c,0.794131,0.199325,0.027116,0.974886,0.005724,0.304118,0.004638,0.006662,0.18579,0.011693,...,0.014357,0.164939,0.009401,0.007832,0.000957,0.395966,0.009054,0.006943,0.001446,0.0
0001812036f1558332e5c0880ecbad70b13a6f28ab04a8db6d83a26ef40aadb0,0.386107,0.117855,0.862294,0.023567,0.274521,0.820201,0.005433,0.749541,0.18579,0.387932,...,0.014357,0.164939,0.005706,0.009877,0.003770,0.395966,0.002888,0.009148,0.001400,1.0
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,0.972260,0.005031,0.005157,0.816084,0.003508,0.188652,0.005562,0.006415,0.18579,0.013120,...,0.014357,0.164939,0.009010,0.007145,0.005159,0.395966,0.000031,0.004060,0.000315,0.0
00057576e6eab4633ec2893ca7e0ab76f2094ad2d43b1e3749db49d51e064ee9,0.080502,0.673698,0.105436,0.127109,0.523456,0.140852,0.455548,0.245338,0.18579,0.140496,...,0.014357,0.164939,1.001770,0.006841,0.979688,0.717009,1.009697,0.003155,0.733949,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff84f243230b1b6b65020994e2c37c196050c21e92e03422aa0c66c5d01c84d,0.519669,0.218172,0.220397,0.185407,0.005225,0.098243,0.004803,0.435218,0.18579,0.097019,...,0.014357,0.164939,0.001928,0.009422,0.008127,0.395966,0.008111,0.008643,0.004765,1.0
fff94ae882244cbe97416913a5ee734eccbc81390db6c15830f89e71c50e3f97,0.853581,0.039299,0.029193,0.917468,0.004437,0.135957,0.004178,0.009728,0.18579,0.159217,...,0.014357,0.164939,0.006014,0.008984,0.007000,0.395966,0.004756,0.005134,0.000925,0.0
fffc59f2e1aa4de05de451121a35c26c61e63cfc38328a6c3500d1a4c43ef317,0.912961,0.005264,0.006156,0.815343,0.004429,0.233056,0.005672,0.006374,0.18579,0.089115,...,0.014357,0.164939,0.009665,0.009193,0.004957,0.395966,0.006643,0.000382,0.007155,0.0


In [12]:
X = df.loc[:,df.columns != 'target'].values
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

# Models

In [13]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns').sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns').sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

def evaluate_metric(X_test, y_test, model):
    ypred = pd.DataFrame(model.predict(X_test), columns = ['prediction'])
    ytest = pd.DataFrame(y_test, columns = ['target'])
    return amex_metric(y_true = ytest, y_pred =ypred)

In [14]:
summary = pd.DataFrame()

### Logistic Regression

In [15]:
lr = LogisticRegression(random_state=0, class_weight= "balanced", n_jobs=-1, max_iter = 500, solver='saga')
parameters = {'penalty' : ['l2', 'none']}
lr_gs = GridSearchCV(lr, parameters, n_jobs=-1)
lr_gs.fit(X_train,y_train)
print("Finalizado entrenamiento")
metric_lr = evaluate_metric(X_test, y_test, lr_gs)
print(lr_gs.best_params_)
print(metric_lr)

/Users/paofernandez/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/paofernandez/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/paofernandez/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/paofernandez/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/paofernandez/opt/anaconda3/lib/python3.9/site-pac

Finalizado entrenamiento
{'penalty': 'l2'}
0.5211383977784526


In [16]:
print(classification_report(y_test, lr_gs.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.96      0.88      0.92      8535
         1.0       0.72      0.90      0.80      2938

    accuracy                           0.88     11473
   macro avg       0.84      0.89      0.86     11473
weighted avg       0.90      0.88      0.89     11473



### Random Forest


In [17]:
rf = RandomForestClassifier( random_state=0, class_weight= "balanced", criterion= 'gini')
parameters = {
        'n_estimators': [100,200,300],
        'max_depth' : [4,5,6,7,8]
        }
rf_gs = GridSearchCV(rf, parameters, n_jobs=-1)
rf_gs.fit(X_train,y_train)
print("Finalizado entrenamiento")
metric_rf = evaluate_metric(X_test, y_test, rf_gs)
print(rf_gs.best_params_)
print(metric_rf)
print(classification_report(y_test, rf_gs.predict(X_test)))

Finalizado entrenamiento
{'max_depth': 8, 'n_estimators': 200}
0.4933419112728931
              precision    recall  f1-score   support

         0.0       0.96      0.86      0.91      8535
         1.0       0.68      0.90      0.78      2938

    accuracy                           0.87     11473
   macro avg       0.82      0.88      0.84     11473
weighted avg       0.89      0.87      0.87     11473



### GradientBoosting

In [18]:
gb = GradientBoostingClassifier(random_state=0)
parameters = {
        'learning_rate': [0.1,0.5,1],
        }
gb_gs = GridSearchCV(gb, parameters, n_jobs=-1)
gb_gs.fit(X_train,y_train)
print("Finalizado entrenamiento")
metric_gb = evaluate_metric(X_test, y_test, gb_gs)
print(metric_gb)
print(classification_report(y_test, gb_gs.predict(X_test)))

Finalizado entrenamiento
0.5618034815916566
              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93      8535
         1.0       0.80      0.79      0.80      2938

    accuracy                           0.90     11473
   macro avg       0.86      0.86      0.86     11473
weighted avg       0.90      0.90      0.90     11473



In [19]:
print(gb_gs.best_params_)

{'learning_rate': 0.1}


In [22]:
filename = 'model_gb.sav'
pickle.dump(gb_gs, open(filename, 'wb'))

In [23]:
filename = 'model_rf.sav'
pickle.dump(rf_gs, open(filename, 'wb'))

In [24]:
filename = 'model_lr.sav'
pickle.dump(lr_gs, open(filename, 'wb'))

In [25]:
imp

SimpleImputer()

In [26]:
filename = 'imputer.sav'
pickle.dump(imp, open(filename, 'wb'))